In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
! pip install sentencepiece
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as tfhub
from tensorflow.keras.models import Model
import tokenization as t

In [ ]:
#Create a function to a train model
def load_model(model_path):
  """
  Loads a saved model from specified path.
  """
  print(f"Loading saved moodel from : {model_path}")
  # model=tf.keras.models.load_model(model_path, custom_objects={"KerasLayer":hub.KerasLayer})
  model=tf.keras.models.load_model(model_path,custom_objects={'KerasLayer':tfhub.KerasLayer})

  return model

In [ ]:
#load a trained  model 
bert_model=load_model("/content/drive/MyDrive/Colab Notebooks/SOAI/pkl_data/20210819-104137-Bert_ass.h5")

Loading saved moodel from : /content/drive/MyDrive/Colab Notebooks/SOAI/pkl_data/20210819-104137-Bert_ass.h5


In [ ]:
def test_tokens(text):
  T=[]
  M=[]
  S=[]
  tokens=tokenizer.tokenize(text)
  if len(tokens) > max_seq_length-2:
    tokens=tokens[0:(max_seq_length-2)]
  if len(tokens)< max_seq_length-2:
    dif=(max_seq_length-2)-len(tokens)
  for j in range(dif):
    tokens=[*tokens,'[PAD]']
  tokens=['[CLS]',*tokens,'[SEP]']
  #mask
  ms=[]
  mask=tokenizer.convert_tokens_to_ids(tokens)
  T.append(mask)
  for j in mask:
    if j>0:
      ms.append(1)
    else:
      ms.append(0)
  M.append(ms)
    
    # segment
  z=[]
  S.append(np.zeros(max_seq_length))
  return np.asarray(T,dtype=float),np.asarray(M,dtype=float),np.asarray(S)

In [ ]:
text_to_test="It is a happy sunny day "

In [ ]:
def replacehtml(x):
  x=x.replace("<+[\w+]+>","")
  return x

In [ ]:
max_seq_length = 60
test_token = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="test_token")
test_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="test_mask")
test_segment = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="test_segment")

In [ ]:
#bert layer 
bert_layer = tfhub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([test_token, test_mask, test_segment])
bert_model_machine = Model(inputs=[test_token, test_mask, test_segment], outputs=pooled_output)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=t.FullTokenizer(vocab_file,do_lower_case)

In [ ]:
preprocess_text=replacehtml(text_to_test)

In [ ]:
test_token, test_mask ,test_segment=test_tokens(preprocess_text)

In [ ]:
# bert_model_machine = load_model("/content/drive/MyDrive/Colab Notebooks/SOAI/pkl_data/20210821-145347-demo_bert.h5")

In [ ]:
X_test_pooled_output=bert_model_machine.predict([test_token, test_mask ,test_segment])

In [ ]:
bert_prediction=bert_model.predict(X_test_pooled_output)

In [ ]:
cls=bert_prediction.argmax(axis=-1)
proab=bert_prediction[0][cls]

In [ ]:
if cls==1:
  print(f"The review is \n ***{text_to_test}***\n The model predcited that the review is POSITIVE and probablity is {proab} ")
else:
  print(f"The review is \n ***{text_to_test}***\n The model predcited that the review is Negative and probablity is {proab} ")

The review is 
 ***It is a happy sunny day ***
 The model predcited that the review is POSITIVE and probablity is [0.9993554] 
